# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
#!pip install pandas

In [6]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [7]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
!pip install langchain_community

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=1)


In [10]:
prompt = ChatPromptTemplate.from_template(
    "What is a {product}? Provide a detailed description of this product including its main features, uses, and benefits."
)

In [11]:

chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/yj/_2dcl_l16cl6dr1bp7h1j6hw0000gn/T/ipykernel_98852/546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [12]:
product = "Queen Size Sheet Set"  # Using the first product from the DataFrame
chain.run(product)

/var/folders/yj/_2dcl_l16cl6dr1bp7h1j6hw0000gn/T/ipykernel_98852/3028145918.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'A Queen Size Sheet Set is a bedding product that typically includes one fitted sheet, one flat sheet, and two standard-size pillowcases designed to fit a queen-size bed. The main feature of a Queen Size Sheet Set is its size which fits a standard queen-size mattress, which typically measures 60 inches wide and 80 inches long.\n\nThe fitted sheet is designed to cover the mattress snugly, ensuring a secure fit during sleep to prevent slipping or coming undone. The flat sheet goes on top of the fitted sheet and is tucked in around the sides and bottom of the mattress for a neat and tidy look. The pillowcases are designed to fit standard size pillows and complete the overall look of the bedding set.\n\nA Queen Size Sheet Set is commonly made from a variety of materials such as cotton, polyester, microfiber, or a blend of these fabrics. Each material offers different benefits, such as breathability, durability, and softness. The choice of material will depend on personal preference and des

## SimpleSequentialChain

In [13]:
from langchain.chains import SimpleSequentialChain

In [14]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is a {product}? Provide a brief description of this product."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on this product description: {product_description}\n\nWrite a compelling marketing summary in 2-3 sentences that would attract customers to buy this product."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [18]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
A Queen Size Sheet Set typically includes a flat sheet, a fitted sheet, and two pillowcases, all designed to fit a standard queen size bed. These sheets are usually made from soft, comfortable materials such as cotton, linen, or microfiber, and come in a variety of colors and patterns to match any bedroom decor. Queen Size Sheet Sets are a convenient way to ensure a coordinated look for your bed and provide a comfortable and stylish sleeping surface.
Transform your bedroom into a cozy oasis with our luxurious Queen Size Sheet Set. Crafted from soft and breathable materials like cotton and microfiber, these sheets are designed to fit your queen size bed perfectly. Choose from a range of colors and patterns to effortlessly upgrade your bedroom decor while enjoying a comfortable and stylish sleeping experience.

> Finished chain.


'Transform your bedroom into a cozy oasis with our luxurious Queen Size Sheet Set. Crafted from soft and breathable materials like cotton and microfiber, these sheets are designed to fit your queen size bed perfectly. Choose from a range of colors and patterns to effortlessly upgrade your bedroom decor while enjoying a comfortable and stylish sleeping experience.'

In [20]:
# Try with a different product from your DataFrame
product2 = df.Product[1]  # "Waterproof Phone Pouch"
overall_simple_chain.run(product2)



> Entering new SimpleSequentialChain chain...
A waterproof phone pouch is a protective case that is designed to keep your phone safe from water damage. It is made of a waterproof material that seals out water, preventing it from entering the pouch and damaging your phone. The pouch typically features a transparent cover that allows you to still use your phone's touchscreen while keeping it protected from water, dirt, sand, and other elements. This accessory is ideal for outdoor activities, water sports, or simply for everyday use to protect your phone from accidental spills or rain.
Don't let water ruin your phone! Keep it safe and dry with our waterproof phone pouch. Whether you're hitting the beach, going kayaking, or just braving a rainy day, this durable and transparent pouch will keep your phone protected and fully functional. Don't leave home without it!

> Finished chain.


"Don't let water ruin your phone! Keep it safe and dry with our waterproof phone pouch. Whether you're hitting the beach, going kayaking, or just braving a rainy day, this durable and transparent pouch will keep your phone protected and fully functional. Don't leave home without it!"

**Repeat the above twice for different products**

## SequentialChain

In [19]:
from langchain.chains import SequentialChain

In [21]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English if it's in another language. If it's already in English, return it as is:\n\n{Review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="english_review"  # Give a name to your output
                    )

In [22]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in 1-2 sentences, highlighting the main points and overall sentiment:\n\n{english_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"  # give a name to this output
                    )

In [23]:
# prompt template 3: detect the language of the review
third_prompt = ChatPromptTemplate.from_template(
    "What language is this review written in? Just return the language name (e.g., 'English', 'Spanish', 'French'):\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [24]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Based on this review summary: {summary}\nAnd knowing the original language was: {language}\n\nWrite a personalized follow-up message to the customer thanking them for their review and addressing their main concerns."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [25]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["english_review", "summary", "followup_message"],
    verbose=True
)

In [39]:
review = df.Review[5]
overall_chain(review)

review2 = df.Review[1]  # "Waterproof Phone Pouch" review
overall_chain({"Review": review2})

review3 = df.Review[2]  # "Luxury Air Mattress" review  
overall_chain({"Review": review3})



> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


{'Review': "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.",
 'english_review': "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattre

**Repeat the above twice for different products or reviews**

## Router Chain

In [27]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [28]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [29]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [30]:
llm = ChatOpenAI(temperature=0)

In [31]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [32]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [33]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [34]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [35]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/yj/_2dcl_l16cl6dr1bp7h1j6hw0000gn/T/ipykernel_98852/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [36]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [37]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [38]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [42]:
# Complete demo of all chain types

# 1. LLM Chain
test_product = df.Product[3]
llm_result = chain.run(test_product)
print("LLM Chain result:", llm_result)

# 2. Simple Sequential Chain  
test_product2 = df.Product[4]
simple_seq_result = overall_simple_chain.run(test_product2)
print("\nSimple Sequential Chain result:", simple_seq_result)

# 3. Sequential Chain
test_review = df.Review[2]
seq_result = overall_chain({"Review": test_review})
print("\nSequential Chain result:", seq_result)

# 4. Router Chain
test_questions = [
    "What is photosynthesis?", 
    "Calculate the integral of 2x",
    "When did the American Civil War end?"
]

for question in test_questions:
    result = chain.run(question)
    print(f"\nRouter Chain - '{question}': {result}")



> Entering new MultiPromptChain chain...
None: {'input': 'Pillows Insert'}
> Finished chain.
LLM Chain result: Pillows inserts are the inner cushion or padding that goes inside a pillow cover. They come in various shapes and sizes to fit different pillow covers and provide support and comfort. Pillow inserts can be made from a variety of materials such as polyester, down, or foam, depending on the desired firmness and softness. They can be easily removed and replaced to keep pillows looking and feeling fresh.


> Entering new SimpleSequentialChain chain...
A milk frother handheld is a small, portable device that is used to froth milk for coffee, lattes, cappuccinos, and other beverages. It typically consists of a wand-like device with a small whisk at the end that spins rapidly to create frothy, airy milk. The handheld nature of the frother allows for easy use and storage, making it a convenient tool for creating cafe-style drinks at home.
Elevate your at-home coffee experience with 